# Import netcdf files

In [1]:
from netCDF4 import Dataset
#import pandas as pd
import numpy as np
#import numpy.ma as ma
import vtk
from vtk.util import numpy_support
from pyproj import Transformer, transform
#from vtk.numpy_interface import dataset_adapter as dsa

In [2]:
src_nc = Dataset("../02_data/nasa/3B-DAY.MS.MRG.3IMERG.20190701-S000000-E235959.V06.nc4.SUB.nc4",
                 mode = "r", format = "NETCDF4")

In [3]:
print(src_nc.data_model)

NETCDF4


In [4]:
print(src_nc.groups)

OrderedDict()


In [5]:
print(src_nc.dimensions)

OrderedDict([('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 1
), ('bnds', <class 'netCDF4._netCDF4.Dimension'>: name = 'bnds', size = 2
), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 20
), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 16
)])


In [29]:
print(src_nc.variables)

OrderedDict([('time', <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    bounds: time_bnds
    units: days since 1970-01-01 00:00:00Z
    calendar: standard
    axis: T
unlimited dimensions: time
current shape = (1,)
filling off
), ('time_bnds', <class 'netCDF4._netCDF4.Variable'>
float64 time_bnds(time, bnds)
unlimited dimensions: time
current shape = (1, 2)
filling off
), ('lon', <class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    standard_name: longitude
    long_name: longitude
    units: degrees_east
    axis: X
unlimited dimensions: 
current shape = (20,)
filling off
), ('lat', <class 'netCDF4._netCDF4.Variable'>
float64 lat(lat)
    standard_name: latitude
    long_name: latitude
    units: degrees_north
    axis: Y
unlimited dimensions: 
current shape = (16,)
filling off
), ('precipitationCal', <class 'netCDF4._netCDF4.Variable'>
float32 precipitationCal(time, lon, lat)
    long_name: Daily accumulated precipitation (combined microwave-IR) e

In [7]:
print(src_nc.ncattrs)

<built-in method ncattrs of netCDF4._netCDF4.Dataset object at 0x7fe70a9d3228>


In [54]:
src_nc.variables["precipitationCal"][:].filled()

array([8.0084692e-07, 2.5012869e-02, 3.8388162e-03, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.4490506e-01,
       3.0113983e+00, 2.2706385e+00, 2.9638443e+00, 3.3354714e+00,
       2.4354711e+00, 7.5571722e-01, 3.9315677e-01, 5.5684385e+00,
       1.7773388e-01, 1.6428895e-01, 1.9686930e-03, 3.2861922e-11,
       0.0000000e+00, 0.0000000e+00, 9.9530678e-07, 4.7502536e-01,
       2.9417477e+00, 1.2586495e+00, 3.1949914e+00, 4.5577235e+00,
       9.6342063e-01, 1.1955750e+00, 2.1562783e-02, 9.6170549e+00,
       2.7640001e-03, 2.3878706e-03, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 5.8589041e-02, 6.8694520e-01,
       2.6410556e+00, 7.1395350e-01, 2.4720936e+00, 1.1586564e+00,
       5.6533951e-01, 5.2120829e-01, 1.6272312e+00, 6.2861991e+00,
       6.8632476e-03, 3.6054271e-06, 1.5992410e-06, 4.9298925e-09,
       0.0000000e+00, 0.0000000e+00, 4.7589830e-01, 6.7122084e-01,
       3.0494201e+00, 7.6080477e-01, 1.7607222e+00, 7.5642884e

In [14]:
# access data of src_nc
# src.variables["var_name"][time][x][y]
print(len(src_nc.variables["precipitationCal"][0]))

20


## Get variable data

### Get coordinates 

* this netcdf file uses coordinates latitude and longitude
* both coordiantes are also used as dimensions
* each coordiante is represented as a 1D array with lon (20 elements) and lat (16 elements)
* this creates a grid of 20*16 elements
* each element has a value for one of the variables HQprecipitation and precipitationCal
* get lat and lon out

In [102]:
lat_dat = src_nc.variables["lat"][:]
lat_dat = lat_dat.filled()
lat_dat

array([54.75, 54.25, 53.75, 53.25, 52.75, 52.25, 51.75, 51.25, 50.75,
       50.25, 49.75, 49.25, 48.75, 48.25, 47.75, 47.25])

In [103]:
lon_dat = src_nc.variables["lon"][:]
lon_dat = lon_dat.filled()

### Get time

In [22]:
time = src_nc.variables["time"]
time

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    bounds: time_bnds
    units: days since 1970-01-01 00:00:00Z
    calendar: standard
    axis: T
unlimited dimensions: time
current shape = (1,)
filling off

In [23]:
time.units

'days since 1970-01-01 00:00:00Z'

In [26]:
# get time data
time = time[:].filled()
time

array([18078.])

### Get variable data

In [30]:
var_names = ["HQprecipitation", "precipitationCal"]
var_data = [None] * len(var_names)
src_vars = np.array([var_names, var_data]).T

In [31]:
# get var_data
for i in range(len(src_vars)):
   src_vars[i][1] = src_nc.variables[(src_vars[i][0])].filled()

In [33]:
src_vars[0]

array(['HQprecipitation',
       array([[[0.00000000e+00, 2.50078179e-02, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.07996881e+00, 1.05505943e+00, 6.89986289e-01, 5.30045867e-01,
         0.00000000e+00, 0.00000000e+00, 6.48440391e-06, 7.00052008e-02],
        [1.09126042e-11, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         2.90052772e-01, 5.00454828e-02, 8.14991951e-01, 8.35025370e-01,
         1.06804682e-05, 4.80012029e-01, 2.74443388e-04, 1.36503148e+00],
        [5.72153283e-07, 8.58233216e-06, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.15005001e-01,
         8.94983292e-01, 1.04894491e-06, 6.24995768e-01, 3.90008628e-01,
         7.99977109e-02, 2.49996990e-01, 2.64986306e-01, 2.39984825e-01],
        [0.00000000e+00, 3.43291958e-06, 0.00000000e+00, 0.00000000e+00,
         0.0000

## Transfrom to vtkPolyData

The type vtkPolydata can use the cell coordinates of the NetCDF file as points and vertice cells. Each point represents, therefore, a cells as well.

### Def vtkPolydata

In [110]:
len(src_vars[0,1][:].flatten())
len(lon_dat.flatten())

320

In [107]:
if (len(lon_dat.flatten()) < len(src_vars[0,1][:].flatten())):
    r = len(src_vars[0,1][:].flatten())/len(lon_dat.flatten())
    lon_dat = np.tile(lon_dat.flatten(), [int(r),1])

In [108]:
if (len(lat_dat.flatten()) < len(src_vars[0,1][:].flatten())):
    r = len(src_vars[0,1][:].flatten())/len(lat_dat.flatten())
    lat_dat = np.repeat(lat_dat.flatten(), int(r))

In [109]:
# def point array
points = np.array([lon_dat.flatten(), 
                   lat_dat.flatten(),
                   np.repeat(0, len(lat_dat.flatten()))
                ]).transpose()
len(points)

320

In [111]:
# def and fill vtkPoints
src_points = vtk.vtkPoints()
src_cells = vtk.vtkCellArray()
#src_points.SetNumberOfPoints(len(points))

# define single point object
#p = np.empty(3, dtype=np.float64)
#p[2] = 0

# def coordinate transformer
transf = Transformer.from_crs(
                        "EPSG:4326",
                        "EPSG:5684",
                        always_xy=True)

for i in range(len(points)):
    p = transf.transform(points[i][0], points[i][1])
    ind = src_points.InsertNextPoint(p[0], p[1], 0)
    src_cells.InsertNextCell(1)
    src_cells.InsertCellPoint(ind)

In [112]:
src_points.GetNumberOfPoints()

320

In [19]:
#cells = np.array([[1]*src_points.GetNumberOfPoints(),
#                 range(src_points.GetNumberOfPoints())
#                 ]).transpose()
#cells = np.ascontiguousarray(cells, dtype=np.int64)
#cells
#src_cells = vtk.vtkCellArray()
#src_cells.SetCells(src_points.GetNumberOfPoints(),
#                   numpy_support.numpy_to_vtkIdTypeArray(cells, deep=True))  

In [113]:
# create vtkPolydata
src_poly = vtk.vtkPolyData()
# be aware to set directions as x-y --> lon-lat not lat-lon
#src_poly.SetDimensions(lat_dat.shape[1], lat_dat.shape[0],1)
src_poly.SetPoints(src_points)

# define vertice based cells
src_poly.SetVerts(src_cells)

### Add data

In [115]:
def add_data_to_src_poly(src_poly, time, src_vars):
    
    for i in range(len(time)):
        for j in range(len(src_vars)):
            arr_name = src_vars[j][0] + "_%s" % str(int(time[i]))
            # remember to transpose the array as the file the netcdf array are defiend by lat-lon (y-x)
            new_point_arr_vtk = numpy_support.numpy_to_vtk(src_vars[j][1][i].T.flatten())
            new_point_arr_vtk.SetName(arr_name)
            src_poly.GetPointData().AddArray(new_point_arr_vtk)
            
            
    src_poly.Modified()

In [116]:
add_data_to_src_poly(src_poly, time, src_vars)

#### Write test file

In [98]:
def write_netcdf_as_vtp(src_obj, outputfile_name):
    write_ouput = vtk.vtkXMLPolyDataWriter()
    write_ouput.SetInputData(src_obj)
    write_ouput.SetFileName(outputfile_name)
    write_ouput.Write() 

In [117]:
write_netcdf_as_vtp(src_poly, "dummy_nasa_precip.vtp")

## Map data on OGS-VTU

### Read OGS-VTU

In [118]:
def read_ogs_vtu(in_filepath):
    dst = vtk.vtkXMLUnstructuredGridReader()
    dst.SetFileName(in_filepath)
    dst.Update()
    return(dst.GetOutput())

In [126]:
ogs_src_vtu = read_ogs_vtu("../02_data/ElbeDomainMesh.vtu")
ogs_src_vtu

(vtkCommonDataModelPython.vtkUnstructuredGrid)0x7fe708ccde28

### Interpolate data on OGS-mesh

In [120]:
def int_kernel():
    # choose your interpolation kernel
    
    # gaussian kernel
    #int_kernel = vtk.vtkGaussianKernel()
    #int_kernel.SetSharpness(2)
    #int_kernel.SetRadius(4000)
    
    #voronoi -- good for categorial data
    int_kernel = vtk.vtkVoronoiKernel()
    
    # shepard kernel
    #int_kernel = vtk.vtkShepardKernel()
    #int_kernel.SetPowerParameter(2)
    #int_kernel.SetRadius(4000)

    return int_kernel


In [121]:
def map_data_on_ogs_vtu(src_poly, dst):
    interpolator = vtk.vtkPointInterpolator()
    interpolator.SetInputData(dst)
    interpolator.SetSourceData(src_poly)
    # def interpolation algorithm
    interpolator.SetKernel(int_kernel())
    # def value if interpolation does not work
    interpolator.SetNullValue(-9999)
    interpolator.Update()
    return(interpolator.GetOutput())

In [127]:
out_vtu = map_data_on_ogs_vtu(src_poly, ogs_src_vtu)
out_vtu

(vtkCommonDataModelPython.vtkUnstructuredGrid)0x7fe708ccd2e8


### Output interpolated data

In [123]:
def write_mapped_ogs_vtu(out_vtu, out_filename):
    write_output = vtk.vtkXMLUnstructuredGridWriter()
    write_output.SetFileName(out_filename)
    write_output.SetInputData(out_vtu)
    write_output.Write()

In [128]:
write_mapped_ogs_vtu(out_vtu, "nasa_precip_elbe.vtu")